In [ ]:
import heapq
import itertools
import math

<div class="alert alert-block alert-info">
<b>Idea for solving:</b>

This problem can be solved using Dijkstra's algorithm for shortest path. The "distance" we want to minimize is the cost of moving amphipods, and the "path" is the moves we have to make to get from the initial configuration to the target configuration.
    
There's 7 positions in the hallway and four rooms with amphipods. We can represent the current position of all the amphipods as a string: The target configuration would be `.......ABCDABCD`. These strings will be our states/nodes in the graph.

The initial configuration is our start node. From this node, we have several possible moves that brings us to another configuration (node) with a certain cost (edge). Every new node we discover has zero or more new nodes attached to it.
    
It does *not* make sense to go back to a previously visited state, as the total cost will be higher than the first time we visited it, and the amphipod configuration will be the same.
    
We keep exploring this graph, using Dijkstras algorithm, and adding new nodes as we go. Eventually, we will end up at the target node, and we will have calculated the lowest possible cost of getting there.
</div>

When calculating possible moves and distances, we transform the state string into a 2D map with row- and column-coordinates.

In [ ]:
hallway_cols = (0, 1, 3, 5, 7, 9, 10)
target_cols = {"A": 2, "B": 4, "C": 6, "D": 8}
energy_cost = { "A": 1, "B": 10, "C": 100, "D": 1000 }

def print_state(state):
    print(f"{state[:2] }.{'.'.join(state[2:5])}.{state[5:7]}")
    print(" ", *state[7:11], " \n ", *state[11:15], " ")
    if len(state) > 15:
        # For part 2
        print(" ", *state[15:19], " \n ", *state[19:], " ")

def get_rooms(state: str): 
    hallway = {(0, col): a for col, a in zip(hallway_cols, state[:7])}
    rooms = {(row, col): a for (row, col), a in zip(itertools.product([1,2,3,4], [2,4,6,8]), state[7:])}
    return hallway | rooms

def get_state(rooms):
    return("".join(rooms.values()))

def distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def allowed_moves(start, rooms):
    moving_type = rooms[start]
    start_row, start_col = start
    target_col = target_cols[moving_type]

    def can_get_to_target_col(start_col, target_col):
        left, right = min(start_col, target_col), max(start_col, target_col)
        between = [col for col in hallway_cols if col > left and col < right]
        return all(rooms[(0, col)] == "." for col in between)

    def all_below_are_solved(row):
        values_below = [rooms.get((below, target_col), "*") for below in range(row + 1, 5)]
        return all(value in "*" + moving_type for value in values_below)
    
    # Starting from a room:
    if start_row > 0:
        # If we're in the correct column and all below are also solved:
        # There is no point in moving.
        if (start_col == target_col) and all_below_are_solved(start_row):
            return []
        # We can move up if the above position is open.
        # We can then move left and right in the hallway until blocked.
        if rooms.get((start_row - 1, start_col), ".") == ".":
            left_cols = [col for col in reversed(hallway_cols) if col < start_col]
            right_cols = [col for col in hallway_cols if col > start_col]
            left_options = itertools.takewhile(lambda col: rooms[0, col] == "." , left_cols)
            right_options = itertools.takewhile(lambda col: rooms[0, col] == "." , right_cols)
            return [(0, col) for col in itertools.chain(left_options, right_options)]

    # Starting from the hallway: Can we get from here to the target column?
    if (start_row == 0) and can_get_to_target_col(start_col, target_col):
        # Is there an open slot in the target column with all below it solved?
        for row in range(1, 5):
            if rooms.get((row, target_col)) != ".":
                break
            if all_below_are_solved(row):
                return [(row, target_col)]

    # If we got here, we can't move anywhere
    return []

def new_possible_states(state):
    rooms = get_rooms(state)
    pod_positions = [pos for pos, pod in rooms.items() if pod in "ABCD"]
    possible_states = []
    for start in pod_positions:
        for end in allowed_moves(start, rooms):
            cost = distance(start, end) * energy_cost[rooms[start]]
            new_rooms = rooms.copy()
            new_rooms[end], new_rooms[start] = new_rooms[start], new_rooms[end]
            possible_states.append((cost, get_state(new_rooms)))

    return possible_states

In [ ]:
def find_lowest_cost(initial_state, target_state):

    queue = [(None, initial_state)]
    cost = {initial_state: 0}
    visited = set((initial_state,))

    while queue:
        _, cur_state = heapq.heappop(queue)
        
        if cur_state == target_state:
            return cost[cur_state]
        
        for extra_cost, new_state in new_possible_states(cur_state):
            cost_candidate = cost[cur_state] + extra_cost
            if cost_candidate < cost.get(new_state, math.inf):
                cost[new_state] = cost_candidate
            if new_state not in visited:
                heapq.heappush(queue, (cost[new_state], new_state))
                visited.add(new_state)

# Part 1

In [ ]:
initial_state = ".......BBDDCCAA"
target_state = ".......ABCDABCD"

assert find_lowest_cost(".......BCBDADCA", target_state) == 12521

In [ ]:
find_lowest_cost(initial_state, target_state)

# Part 2

In [ ]:
initial_state = ".......BBDDDCBADBACCCAA"
target_state = ".......ABCDABCDABCDABCD"

assert find_lowest_cost(".......BCBDDCBADBACADCA", target_state) == 44169

In [ ]:
find_lowest_cost(initial_state, target_state)